In this We have taken from 2021-01-01 previous it wa 2021-09-01 for the start date of the in the train variable of the Model

And the forecasting is done till July 2024

In [1]:
import pandas as pd
import numpy as np


# for Modelling
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, GRU, LSTM, Dense

from tensorflow.keras.models import load_model

# for Evaluation
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.metrics import mean_absolute_error, r2_score


In [2]:
df= pd.read_csv('ax_df_all.csv')
df

,Time id,Material,Material_Id,Quantity,Total_Value
0,9,E11-200-001,34342,13,1048203.00
1,10,E11-200-001,34342,28,2257668.00
2,11,E11-200-001,34342,38,2978019.06
3,12,E11-200-001,34342,42,3275484.94
4,13,E11-200-001,34342,32,2519632.00
...,...,...,...,...,...
3124,34,C04-200123,146,212,108739.70
3125,35,C04-200123,146,252,128604.24
3126,36,C04-200123,146,254,128352.64
3127,37,C04-200123,146,238,119471.66


In [3]:
# Define a dictionary to map Time_id to dates
time_id_to_date = {
    1: "2021-01-01", 2: "2021-02-01", 3: "2021-03-01", 4: "2021-04-01", 5: "2021-05-01",
    6: "2021-06-01", 7: "2021-07-01", 8: "2021-08-01", 9: "2021-09-01", 10: "2021-10-01",
    11: "2021-11-01", 12: "2021-12-01", 13: "2022-01-01", 14: "2022-02-01", 15: "2022-03-01",
    16: "2022-04-01", 17: "2022-05-01", 18: "2022-06-01", 19: "2022-07-01", 20: "2022-08-01",
    21: "2022-09-01", 22: "2022-10-01", 23: "2022-11-01", 24: "2022-12-01", 25: "2023-01-01",
    26: "2023-02-01", 27: "2023-03-01", 28: "2023-04-01", 29: "2023-05-01", 30: "2023-06-01",
    31: "2023-07-01", 32: "2023-08-01", 33: "2023-09-01", 34: "2023-10-01", 35: "2023-11-01",
    36: "2023-12-01", 37: "2024-01-01", 38: "2024-02-01", 39: "2024-02-01"
}

# Change Time_id column to dates using the dictionary
df['Time id'] = df['Time id'].map(time_id_to_date)

# Optionally, convert the Time_id column to datetime type
df['Time id'] = pd.to_datetime(df['Time id'])

In [4]:
data = df.copy()

In [5]:
materials = ['C87-200433', 'S02-060380', 'S02-060352',
       'S01-070021', 'C87-200349', 'C87-400114', 'S01-110220',
       'S02-010175', 'S03-030109', 'S02-060363', 'E58-200-001',
       'S02-060362', 'C87-200560', 'S02-060446', 'E51-200-001',
       'S10-130517', 'SA-A46-07-0100E', 'S02-060445', 'C87-300025',
       'S10-130761', 'C72-100172', 'S25-050054', 'S02-060313',
       'S01-090022', 'S02-060325', 'S06-380001', 'E46-200-001',
       'S10-130309', 'S25-010196', 'S26-090085', 'S10-130652',
       'C04-400146', 'S02-060315', 'S26-090134', 'C87-000231',
       'C87-200463', 'C87-300008', 'C37-300179', 'S06-080061',
       'C90-200051', 'S10-130277', 'C72-000163', 'S12-060213',
       'C60-700112', 'S02-060364', 'S25-010204-01', 'C06-600290',
       'S21-001-320', 'S06-020222', 'S02-060354', 'C37-200275',
       'S12-060139', 'S03-070005', 'C87-400124', 'C87-200403',
       'C04-400148', 'S27-010112', 'S07-471173', 'SA-A07-07-0109',
       'C04-400111', 'S06-080063', 'S02-020065', 'C37-200258',
       'S25-050051', 'S06-070046', 'S06-040109', 'H17-200228',
       'C70-300037', 'C29-300110', 'S10-130616', 'S10-130615',
       'S06-360116', 'C65-200280', 'S23-170045', 'AC61-000002',
       'C37-300152', 'C70-300048', 'C37-500421', 'S10-130537',
       'C04-400202', 'C04-400200', 'C07-000704', 'S02-060395',
       'C97-100180', 'C87-100152', 'C65-200279', 'S10-130625',
       'C04-200123']

# Simple RNN, GRU, LSTM

For this analyis Qunatity column is taken into consideration and unique Material is taken for the forecasting since the data we have is from 2021-09-01 to 2024-02-01 so for the forecasting the train set that is taken is from 2021-09-01 to 2023-10-01 and for test 2023-10-01 to 2024-02-01. Then MinMax Normalizatin technique is used to scale the data.

In [6]:
# Define the evaluate_model function
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    return mae, r2

# Convert 'Time id' to datetime
data['Time id'] = pd.to_datetime(data['Time id'])

# Define train and test periods
train_start_date = '2021-01-01'
train_end_date = '2023-10-01'
test_start_date = '2023-11-01'
test_end_date = '2024-02-01'

# Define a function to build and train RNN models
def build_and_train_model(model_type, X_train, y_train):
    model = Sequential()
    if model_type == 'SimpleRNN':
        model.add(SimpleRNN(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    elif model_type == 'GRU':
        model.add(GRU(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    elif model_type == 'LSTM':
        model.add(LSTM(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0)
    return model

# Iterate over each material
for material in data['Material'].unique():
    print(f"Evaluating RNN models for Material: {material}")
    
    # Filter data for the current material
    material_data = data[data['Material'] == material]

    # Split data into train and test sets
    train = material_data[(material_data['Time id'] >= train_start_date) & (material_data['Time id'] <= train_end_date)]
    test = material_data[(material_data['Time id'] >= test_start_date) & (material_data['Time id'] <= test_end_date)]

    # Scale the data
    scaler = MinMaxScaler()
    train_scaled = scaler.fit_transform(train[['Quantity']])
    test_scaled = scaler.transform(test[['Quantity']])

    # Reshape input to be [samples, time steps, features]
    X_train = np.reshape(train_scaled, (train_scaled.shape[0], 1, 1))
    X_test = np.reshape(test_scaled, (test_scaled.shape[0], 1, 1))
    y_train = train_scaled[:, 0]  # Target variable is Quantity
    y_test = test_scaled[:, 0]

    # Build and train Simple RNN model
    simple_rnn_model = build_and_train_model('SimpleRNN', X_train, y_train)

    # Build and train GRU model
    gru_model = build_and_train_model('GRU', X_train, y_train)

    # Build and train LSTM model
    lstm_model = build_and_train_model('LSTM', X_train, y_train)

    # Evaluate models on test set
    simple_rnn_mae, simple_rnn_r2 = evaluate_model(simple_rnn_model, X_test, y_test)
    gru_mae, gru_r2 = evaluate_model(gru_model, X_test, y_test)
    lstm_mae, lstm_r2 = evaluate_model(lstm_model, X_test, y_test)

    print("Simple RNN MAE:", simple_rnn_mae, " R2 Score:", simple_rnn_r2)
    print("GRU MAE:", gru_mae, " R2 Score:", gru_r2)
    print("LSTM MAE:", lstm_mae, " R2 Score:", lstm_r2)
    print("-" * 50)


Evaluating RNN models for Material: E11-200-001


C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 260ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 348ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 299ms/step
Simple RNN MAE: 0.08005077242851252  R2 Score: 0.9160536199490295
GRU MAE: 0.22425012141466136  R2 Score: 0.35284758504565805
LSTM MAE: 0.21713028699159617  R2 Score: 0.3888694854198136
--------------------------------------------------
Evaluating RNN models for Material: C87-200433


C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 278ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 376ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 321ms/step
Simple RNN MAE: 0.0022271884266625713  R2 Score: 0.9995713978518671
GRU MAE: 0.05233805660497058  R2 Score: 0.7606370471623741
LSTM MAE: 0.0917301591147076  R2 Score: 0.2657190712951373
--------------------------------------------------
Evaluating RNN models for Material: S02-060380


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 184ms/step
Simple RNN MAE: 0.02536434956959316  R2 Score: 0.6280368507153492
GRU MAE: 0.042033856681415016  R2 Score: 0.3616507532902644
LSTM MAE: 0.030235594085284645  R2 Score: 0.5063172747500504
--------------------------------------------------
Evaluating RNN models for Material: S10-130625


C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step
Simple RNN MAE: 0.009536887862180404  R2 Score: 0.9673589069871636
GRU MAE: 0.031736028821844814  R2 Score: 0.5899177908283961
LSTM MAE: 0.02042138145158165  R2 Score: 0.7498500924539526
--------------------------------------------------
Evaluating RNN models for Material: C04-200123


C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step
Simple RNN MAE: 0.010324046015739496  R2 Score: 0.9639927424699279
GRU MAE: 0.03978066891431814  R2 Score: 0.4770201952038954
LSTM MAE: 0.038036793470382746  R2 Score: 0.4704071352866356
--------------------------------------------------


Since After the evaluation of all the Materials Now We are forecasting for the next four month using the best mmodel that we that got after the analysis. 

So Simple RNN was giving best accuracy for the r2 and mae metric so we have consider Simple RNN 

In [8]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from datetime import timedelta

# Define the function to build and train the Simple RNN model
def build_and_train_simple_rnn(X_train, y_train):
    model = Sequential()
    model.add(SimpleRNN(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0)
    return model

# Convert 'Time id' to datetime
data['Time id'] = pd.to_datetime(data['Time id'])

# Define the forecasting period
forecast_start_date = data['Time id'].max() + timedelta(days=1)
forecast_end_date = forecast_start_date + timedelta(days=6*30)  # Roughly 6 months
forecast_dates = pd.date_range(start=forecast_start_date, end=forecast_end_date, freq='MS')

# Prepare the DataFrame to hold the forecast results
forecast_df = pd.DataFrame(columns=['Material', 'Time id', 'Forecasted Quantity'])

# Iterate over each material
for material in data['Material'].unique():
    print(f"Forecasting for Material: {material}")
    
    # Filter data for the current material
    material_data = data[data['Material'] == material]
    
    # Use all available data for training
    train = material_data[['Time id', 'Quantity']]
    
    # Scale the data
    scaler = MinMaxScaler()
    train_scaled = scaler.fit_transform(train[['Quantity']])
    
    # Reshape input to be [samples, time steps, features]
    X_train = np.reshape(train_scaled, (train_scaled.shape[0], 1, 1))
    y_train = train_scaled[:, 0]  # Target variable is Quantity
    
    # Build and train Simple RNN model
    simple_rnn_model = build_and_train_simple_rnn(X_train, y_train)
    
    # Forecast the next 6 months
    last_value = train_scaled[-1]
    forecasted_scaled_quantities = []
    
    for _ in range(len(forecast_dates)):
        next_input = np.reshape(last_value, (1, 1, 1))
        next_value_scaled = simple_rnn_model.predict(next_input)[0][0]
        forecasted_scaled_quantities.append(next_value_scaled)
        last_value = next_value_scaled
    
    # Inverse transform the forecasted values
    forecasted_quantities = scaler.inverse_transform(np.array(forecasted_scaled_quantities).reshape(-1, 1)).flatten()
    
    # Create a DataFrame for the current material's forecast
    material_forecast_df = pd.DataFrame({
        'Material': material,
        'Time id': forecast_dates,
        'Forecasted Quantity': forecasted_quantities
    })
    
    # Append the current material's forecast to the overall forecast DataFrame
    forecast_df = pd.concat([forecast_df, material_forecast_df], ignore_index=True)

print("Forecasting complete!")
print(forecast_df)


Forecasting for Material: E11-200-001


C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Forecasting for Material: C87-200433


C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Forecasting for Material: S02-060380


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Forecasting for Material: S10-130625


C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Forecasting for Material: C04-200123


C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Forecasting complete!
        Material    Time id  Forecasted Quantity
0    E11-200-001 2024-03-01            54.335094
1    E11-200-001 2024-04-01            52.047615
2    E11-200-001 2024-05-01            50.613506
3    E11-200-001 2024-06-01            49.712029
4    E11-200-001 2024-07-01            49.144455
..           ...        ...                  ...
440   C04-200123 2024-03-01           259.983246
441   C04-200123 2024-04-01           258.206970
442   C04-200123 2024-05-01           256.640900
443   C04-200123 2024-06-01           255.258987
444   C04-200123 2024-07-01           254.038666

[445 rows x 3 columns]


In [9]:
# Combine the original data with the forecasted data
combined_df = pd.concat([data, forecast_df], ignore_index=True)

# Sort by 'Material' and 'Time id' to maintain a chronological order for each material
combined_df = combined_df.sort_values(by=['Material', 'Time id']).reset_index(drop=True)

print("Combined data with forecasts:")
print(combined_df)


Combined data with forecasts:
        Time id         Material  Material_Id  Quantity  Total_Value  \
0    2021-01-01      AC61-000002         87.0      25.0     100000.0   
1    2021-02-01      AC61-000002         87.0      41.0     164000.0   
2    2021-03-01      AC61-000002         87.0      47.0     188000.0   
3    2021-04-01      AC61-000002         87.0      24.0     100480.0   
4    2021-06-01      AC61-000002         87.0      14.0      60480.0   
...         ...              ...          ...       ...          ...   
3569 2024-03-01  SA-A46-07-0100E          NaN       NaN          NaN   
3570 2024-04-01  SA-A46-07-0100E          NaN       NaN          NaN   
3571 2024-05-01  SA-A46-07-0100E          NaN       NaN          NaN   
3572 2024-06-01  SA-A46-07-0100E          NaN       NaN          NaN   
3573 2024-07-01  SA-A46-07-0100E          NaN       NaN          NaN   

      Forecasted Quantity  
0                     NaN  
1                     NaN  
2                    

Here we have Combine the two dataframe and got the final dataframe that containes historical data of Material and the forecasted  Quantity